# Hyperparameter optimization with RayTunee
**Authorship:**
Adam Klie (last updated: *07/23/2023*)
***
**Description:**
This notebook is meant to serve as a rough guide for performing hyperparameter optimization with RayTune. 

In [163]:
import ray
from ray import tune
import seqdatasets
import torch
from eugene import preprocess as pp
from eugene.train import hyperopt

In [164]:
ray.init(num_cpus=4, num_gpus=1)

2023-07-16 18:24:08,278	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.9.16
Ray version:,2.4.0


# Load data

In [169]:
# Load in some toy data and preproess
sdata = seqdatasets.random1000()
pp.ohe_seqs_sdata(sdata)
pp.make_unique_ids_sdata(sdata)
pp.train_test_random_split(sdata, dim="_sequence")
sdata["ohe_seq"] = sdata["ohe_seq"].transpose("_sequence", "_ohe", "length")
sdata

<xarray.Dataset>
Dimensions:     (_sequence: 1000, _ohe: 4, length: 100)
Dimensions without coordinates: _sequence, _ohe, length
Data variables: (12/25)
    activity_0  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_1  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_2  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_3  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_4  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    activity_5  (_sequence) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    ...          ...
    label_9     (_sequence) int64 dask.array<chunksize=(1000,), meta=np.ndarray>
    name        (_sequence) object dask.array<chunksize=(1000,), meta=np.ndarray>
    seq         (_sequence) object dask.array<chunksize=(1000,), meta=np.ndarray>
    ohe_seq     (_sequence, _ohe, length) uint8 0 0 0 0 1 0 0 ... 1 0 0 0 0 0 0
    id          (_sequence) <U7 'seq0000' 'seq0001' ... 'seq0998' 'seq0999'
    train_val   (_sequence) bool True True True True ... True True True True
Attributes:
    max_jitter:  0

# Setting up the training config

In [170]:
tune_config = {
    "module": "SequenceModule",
    "model": {
    "arch_name": "CNN",
    "model_name": "hyper_funkytiger",
    "arch": {
      "input_len": 100,
      "output_dim": 2,
      "conv_kwargs": {
        "input_channels": 4,
        "conv_channels": [tune.choice([32, 64, 128, 256, 512]), tune.choice([32, 64, 128, 256, 512]), tune.choice([32, 64, 128, 256, 512])],
        "conv_kernels": [tune.choice([3, 5, 7]), tune.choice([3, 5, 7]), tune.choice([3, 5, 7])],
        "pool_kernels": [tune.choice([1, 2]), tune.choice([1, 2]), tune.choice([1, 2])],
        "dropout_rates": 0.1,
        "batchnorm": True
      },
      "dense_kwargs": {
        "hidden_dims": [tune.choice([32, 64, 128, 256, 512]), tune.choice([32, 64, 128, 256, 512])],
      },
    },
    "task": "regression",
    "loss_fxn": "mse",
    "optimizer": "adam",
    "optimizer_lr": tune.loguniform(1e-4, 1e-1),
  },
  "batch_size": tune.choice([32, 64, 128]),
}

# Tuning with the defaults
The default scheduler is the ASHAScheduler from Ray, which is a variant of the HyperBand algorithm. The default search algorithm is the "BasicVariantGenerator" from Ray.

In [178]:
res = hyperopt(
    tune_config,
    sdata=sdata,
    target_vars=["activity_0", "activity_1"],
    train_var="train_val",
    epochs=5,
    transforms={"ohe_seq": lambda x: torch.tensor(x, dtype=torch.float32)},
    gpus=1,
    cpus=4,
    num_samples=5,
    num_workers=0
)

# Tuning with HyperOptSearch algorithm
This time we will use the HyperOptSearch algorithm, 
You can also "BayesOptSearch" as an alternative to HyperOptSearch and "MedianStoppingRule" or "PopulationBasedTraining" as alternatives to ASHA.a


In [ ]:
res = hyperopt(
    tune_config,
    sdata=sdata,
    target_vars=["activity_0", "activity_1"],
    train_var="train_val",
    epochs=5,
    transforms={"ohe_seq": lambda x: torch.tensor(x, dtype=torch.float32)},
    gpus=1,
    cpus=4,
    num_samples=5,
    num_workers=0,
    algorithm="HyperOptSearch",
)

# DONE!

---